In [1]:
%load_ext autoreload
import sys
if not '/Users/nlarusstone/Documents/MPhil/Research/cf_fba' in sys.path:
    sys.path.append('/Users/nlarusstone/Documents/MPhil/Research/cf_fba')
import src.utils as utils
%autoreload 2

In [19]:
import cobra
import cobra.test
import pandas as pd
import numpy as np
import Bio.PDB.Polypeptide

In [17]:
model = cobra.io.read_sbml_model(filename='../models/iJO1366.xml')

In [20]:
varner = cobra.io.load_json_model('../models/varner.json')
def extract_txtl_rxns(model):
    aa_metabs = []
    for aa in Bio.PDB.Polypeptide.aa3:
        aa_metabs += model.metabolites.query(aa.lower())
    aa_rxns = []
    for aa_metab in aa_metabs:
        aa_rxns += aa_metab.reactions
    tx_rxns = model.reactions.query('transcription')
    tl_rxns = model.reactions.query('translation')
    txtl_rxns = list(set(aa_rxns).union(tx_rxns).union(tl_rxns))
    return txtl_rxns
txtl_rxns = extract_txtl_rxns(varner)

In [8]:
alc_dehydr = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='c')
model.add_reaction(alc_dehydr)
alc_dehydr.add_metabolites({'btcoa_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, 'btal_c': 1})

butanol = cobra.Metabolite(id='btol_c', name='1-butanol', compartment='c', charge=0, formula='C4H9OH')
but_synth = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='c')
model.add_reaction(but_synth)
but_synth.add_metabolites({'btal_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, butanol: 1})

In [9]:
utils.change_obj(model=model, metab=model.metabolites.btol_c)

In [10]:
cobra.io.write_sbml_model(filename='../models/ecoli_but.sbml', cobra_model=model)

In [11]:
def ecoli_to_cf(mod):
    # Delete transmembrane transport reactions
    model = mod.copy()

    # Add exchange reactions for all metabolites
    # Exchange‐reactions for non‐substrate metabolites have lb of 0 and ub of 1000 by default
    #all_metab_ex = utils.gen_metab_dict(model, model.metabolites, cnvt=False)

    #utils.add_ex(model, all_metab_ex)
    model.remove_reactions(['EX_glc_e', 'EX_pi_e', 'EX_mg2_e', 'EX_k_e', 'EX_nh4_e'])

    # As objective function, we selected the exchange reaction which corresponds to the target metabolite 
    # for which a pathway should be determined.   
    return model

model_cf = ecoli_to_cf(model)

In [12]:
def build_medium(mod):
    model = mod.copy()
    # in mM
    aa_conc = -1 * utils.conc_to_flux(2)
    trna_conc = -1 * utils.conc_to_flux(0.17)
    metab_conc = map(lambda x: utils.conc_to_flux(x) * -1, [1.2, 0.85, 0.85, 0.85, 0.33, 0.27, 1.50, 1.00, 130, 10, 12, 0.33])
    print metab_conc
    # TODO: map from conc to exchange fluxes
       
    import Bio.PDB.Polypeptide
    aas = map(lambda x: x.lower(), Bio.PDB.Polypeptide.aa3)
    aa_concs = {}
    for aa in aas:
        l_aa, d_aa = None, None
        try:
            l_aa = utils.convert_cmpts(model, model.metabolites.get_by_id(aa + '__L_c'), from_cmpt='c', to_cmpt='e')
            d_aa = utils.convert_cmpts(model, model.metabolites.get_by_id(aa + '__D_c'), from_cmpt='c', to_cmpt='e')
        except KeyError:
            if aa == 'gly':
                l_aa = utils.convert_cmpts(model, model.metabolites.get_by_id(aa + '_c'), from_cmpt='c', to_cmpt='e')
        if d_aa:
            aa_concs[l_aa] = aa_conc / 2.0 
            aa_concs[d_aa] = aa_conc / 2.0 
        else:
            aa_concs[l_aa] = aa_conc
    
    trna_concs = {utils.convert_cmpts(model, trna_metab, from_cmpt='c', to_cmpt='e') : trna_conc for trna_metab in model.metabolites.query('trna')}
       
    metab_nms = ['atp', 'gtp', 'utp', 'ctp', 'nad', 'coa', 'spmd', 'ptrc', 'k', 'nh4', 'mg2', 'pep']
    metab_concs = utils.gen_metab_dict(model, metab_nms, metab_conc)
    
    # Not included: RNAP & Folinic Acid & cell extract
    base_medium = {}
    base_medium.update(utils.add_exchange(model, metab_concs))
    #base_medium.update(utils.add_exchange(model, aa_concs))
    #base_medium.update(utils.add_exchange(model, trna_concs))
       
    #model.medium = base_medium
    return model

In [13]:
model_cf_med = utils.build_medium(model_cf)

In [14]:
cobra.io.write_sbml_model(filename='../models/ecoli_cf_base.sbml', cobra_model=model_cf_med)

In [68]:
model_cf_med.optimize()
model_cf_med.summary()

IN FLUXES        OUT FLUXES        OBJECTIVES
---------------  ----------------  ----------------
o2_e      2.03   h2o_e    2.09     DM_btol_c  0.118
pro__L_e  0.473  co2_e    1.99
glu__L_e  0.417  gdp_e    0.354
h_e       0.39   btol_c   0.118
gtp_e     0.354  ura_e    0.0523
val__L_e  0.118  agm_e    0.0456
cys__L_e  0.118  alltn_e  0.00311


In [24]:
df = pd.read_csv('../data/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

,AVG,STD,AVG.1,STD.1,Mg(Glu)2,NH4(Glu),K(Glu),Glucose,K2HPO4,NAD,ATP,CoA
0,0.00,0.0,0.00,0.00,8.0,10.0,134,0,10,0.5,0.0,0.5
1,0.00,0.0,0.00,0.00,8.0,10.0,134,200,10,0.0,0.0,0.5
2,0.00,0.0,0.00,0.00,8.0,10.0,134,200,10,0.5,0.0,2.0
3,0.00,0.0,0.00,0.00,8.0,10.0,134,200,10,0.5,0.0,1.6
4,1.21,1.7,0.09,0.13,8.0,10.0,134,200,10,0.5,2.0,0.5


In [55]:
reduced = df.loc[:, ['Glucose', 'NH4(Glu)', 'NAD', 'CoA']].drop_duplicates(keep='first')
reduced['AVG.1'] = df.loc[reduced.index, 'AVG.1']
reduced.to_csv('../data/reduced.csv')

In [24]:
utils.change_obj(model=model_cf_med, metab=model_cf_med.metabolites.btol_c)
utils.change_obj(model=model_cf, metab=model_cf.metabolites.btol_c)

In [66]:
def add_addl_reactants(model, df):
    mod = model.copy()
    addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc__D', 'pi', 'nad', 'atp', 'coa']
    objs = []
    for i, row in df.iterrows():
        with mod:
            metab_dict = utils.gen_metab_dict(mod, addl_reagent_nms, map(lambda x: -1 * utils.conc_to_flux(x), row[4:]))
            rxn = utils.add_exchange(mod, metab_dict, additive=True)
            #different_mediums(mod, model)
            #mod.add_reactions(reaction_list=sol[0])
            obj = mod.slim_optimize()
            objs.append(obj)
            #print 'Obj: {0}'.format(obj.objective_value)
    return objs

In [70]:
model.metabolites.mg2_c

Metabolite identifier,mg2_c
Name,magnesium
Memory address,0x0109757410
Formula,Mg
Compartment,c
In 5 reaction(s),"Ec_biomass_iJO1366_core_53p95M, MG2t3_2pp, MG2uabcpp, Ec_biomass_iJO1366_WT_53p95M, MG2tpp"


In [69]:
utils.add_addl_reactants(model_cf_med, df)

[1.0632867132867072,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 2.895833333333339,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 2.340277777777785,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.7847222222222205,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 0.11805555555555675,
 0.6736111111111152,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435,
 1.5069444444444435]

In [12]:
cobra.io.save_json_model(filename='Full_Ecoli_CF_Base.json', model=model_cf_med)

In [81]:
model.medium

{'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [13]:
model.optimize()
model.summary()

IN FLUXES             OUT FLUXES           OBJECTIVES
--------------------  -------------------  ----------------------
o2_e       17.6       h2o_e     45.6       Ec_biomass_i...  0.982
nh4_e      10.6       co2_e     19.7
glc__D_e   10         h_e        9.03
pi_e        0.948     mththf_c   0.00044
so4_e       0.248     5drib_c    0.000221
k_e         0.192     4crsol_c   0.000219
fe2_e       0.0158    amob_c     1.96e-06
mg2_e       0.00852   meoh_e     1.96e-06
ca2_e       0.00511
cl_e        0.00511
cu2_e       0.000697
mn2_e       0.000679
zn2_e       0.000335
ni2_e       0.000317
mobd_e      0.000127
cobalt2_e   2.46e-05


In [17]:
fl = -1 * utils.conc_to_flux(200)
print fl
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

-83.3333333333
IN FLUXES              OUT FLUXES            OBJECTIVES
---------------------  --------------------  ---------------------
o2_e       142         h2o_e     379         Ec_biomass_i...  8.28
nh4_e       89.5       co2_e     160
glc__D_e    83.3       h_e        76.1
pi_e         7.99      mththf_c    0.00371
so4_e        2.09      5drib_c     0.00186
k_e          1.62      4crsol_c    0.00185
fe2_e        0.133     amob_c      1.66e-05
mg2_e        0.0718    meoh_e      1.66e-05
ca2_e        0.0431
cl_e         0.0431
cu2_e        0.00587
mn2_e        0.00572
zn2_e        0.00282
ni2_e        0.00268
mobd_e       0.00107
cobalt2_e    0.000207


In [18]:
fl = -1 * utils.conc_to_flux(120)
print fl
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

-50.0
IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  ---------------------
o2_e       85.6       h2o_e     227         Ec_biomass_i...  4.96
nh4_e      53.6       co2_e      96.2
glc__D_e   50         h_e        45.6
pi_e        4.79      mththf_c    0.00222
so4_e       1.25      5drib_c     0.00112
k_e         0.969     4crsol_c    0.00111
fe2_e       0.0797    amob_c      9.93e-06
mg2_e       0.0431    meoh_e      9.93e-06
ca2_e       0.0258
cl_e        0.0258
cu2_e       0.00352
mn2_e       0.00343
zn2_e       0.00169
ni2_e       0.0016
mobd_e      0.00064
cobalt2_e   0.000124


In [92]:
utils.change_obj(model, model.metabolites.btol_c)

In [20]:
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = -10, 1000
model.reactions.EX_glc_e
model.optimize()
model.summary()

IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ---------------
nh4_e     12.5   h2o_e   47.5   DM_btol_c  1.79
glc__D_e  10     co2_e   15.2
o2_e       9.87  h_e      5.37
pi_e       5.37  btol_c   1.79
so4_e      1.79


In [21]:
fl = -1 * utils.conc_to_flux(200)
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ---------------
nh4_e     106    h2o_e   394    DM_btol_c  15.1
glc__D_e   83.3  co2_e   123
o2_e       77.7  h_e      45.3
pi_e       45.3  btol_c   15.1
so4_e      15.1


In [22]:
fl = -1 * utils.conc_to_flux(120)
glc = model.reactions.EX_glc_e
glc.lower_bound, glc.upper_bound = fl, fl
model.optimize()
model.summary()

IN FLUXES        OUT FLUXES      OBJECTIVES
---------------  --------------  ---------------
nh4_e     63.3   h2o_e   237     DM_btol_c  9.04
glc__D_e  50     co2_e    74
o2_e      46.8   h_e      27.1
pi_e      27.1   btol_c    9.04
so4_e      9.04


In [24]:
fl = -1 * utils.conc_to_flux(200)
fl_nh4 = -1 * utils.conc_to_flux(10)
print fl, fl_nh4
glc = model.reactions.EX_glc_e
nh4 = model.reactions.EX_nh4_e
glc.lower_bound, glc.upper_bound = fl, fl
nh4.lower_bound, nh4.upper_bound = fl_nh4, fl_nh4
model.optimize()
model.summary()

-83.3333333333 -4.16666666667
IN FLUXES         OUT FLUXES         OBJECTIVES
----------------  -----------------  ----------------
glc__D_e  83.3    h2o_e     122      DM_btol_c  0.595
o2_e      61.1    co2_e     101
nh4_e      4.17   h_e        61.7
pi_e       1.79   3hpp_e     28.8
so4_e      0.595  mththf_c   28.8
                  hxa_e      21.6
                  akg_e       4.76
                  btol_c      0.595


In [97]:
fl = -1 * utils.conc_to_flux(200)
fl_nh4 = -1 * utils.conc_to_flux(10)
fl_pi = -1 * utils.conc_to_flux(10)
fl_mg = -1 * utils.conc_to_flux(8)
print fl, fl_nh4
glc = model.reactions.EX_glc_e
nh4 = model.reactions.EX_nh4_e
pi = model.reactions.EX_pi_e
mg = model.reactions.EX_mg2_e
def update_bnds(rxn, fl): 
    rxn.lower_bound, rxn.upper_bound = fl, 0 
glc.lower_bound, glc.upper_bound = fl, fl
nh4.lower_bound, nh4.upper_bound = fl_nh4, fl_nh4
update_bnds(pi, fl_pi)
update_bnds(mg, fl_mg)
model.optimize()
model.summary()

-83.3333333333 -4.16666666667
IN FLUXES         OUT FLUXES           OBJECTIVES
----------------  -------------------  ----------------
glc__D_e  83.3    dha_e       73.1     DM_btol_c  0.595
o2_e      13.6    co2_e       53.2
nh4_e      4.17   12ppd__R_e  42.7
pi_e       1.83   h_e         40.3
so4_e      0.595  ac_e        28.8
                  h2o_e       25.4
                  mal__L_e     4.15
                  glyclt_e     1.48
                  mththf_c     1.48
                  btol_c       0.595
                  glyc3p_e     0.0425


In [103]:
elts = ['mg2', 'nh4', 'k', 'glc', 'pi', 'nadh', 'atp', 'coa']
row = df.iloc[36, 4:]
fluxes = [-1 * utils.conc_to_flux(row[i]) for i in range(len(elts))]
print elts
print row.tolist()
for elt, flux in zip(elts, fluxes):
    #print elt, '{0}_c'.format(elt)#, model.metabolites.get_by_id('{0}_c'.format(elt))
    try:
        rxn = model.reactions.get_by_id('EX_{0}_e'.format(elt))
    except KeyError:
        break
        #model.add_boundary(reaction_id='EX_{0}_e'.format(elt), type='Exchange', 
        #                   metabolite=model.metabolites.get_by_id('{0}_c'.format(elt)), ub=flux)
        #rxn = model.reactions.get_by_id('EX_{0}_e'.format(elt))
    rxn.lower_bound, rxn.upper_bound = flux, 0
model.optimize()
model.summary()

['mg2', 'nh4', 'k', 'glc', 'pi', 'nadh', 'atp', 'coa']
[8.0, 10.0, 134.0, 200.0, 4.0, 0.5, 0.0, 0.5]
IN FLUXES        OUT FLUXES        OBJECTIVES
---------------  ----------------  ----------------
o2_e      6.5    h2o_e     18.6    DM_btol_c  0.556
glc__D_e  4.49   co2_e      7.75
nh4_e     3.89   fe3_e      1.67
fe2_e     1.67   mththf_c   1.06
pi_e      1.67   btol_c     0.556
so4_e     0.556


In [22]:
df2 = pd.read_csv(filepath_or_buffer='../data/4_5_18_T7_C6N_mRFP.TXT', skiprows=6)

In [27]:
df2.columns

Index([u'Well', u'Unnamed: 1', u'C01', u'C02', u'C03', u'C04', u'C05', u'C06',
       u'C07', u'C08', u'C09', u'D01', u'D02', u'D03', u'D04', u'D05', u'D06',
       u'D07', u'D08', u'D09', u'E01', u'E02', u'E03', u'E04', u'E05', u'E06',
       u'E07', u'E08', u'E09', u'F01', u'F02', u'F03', u'F04', u'F05', u'F06',
       u'F07', u'F08', u'F09', u'F10'],
      dtype='object')